Wege um die Effizient der Schachbretterkennugn zu erhöhen:
1. Extrapolation
2. SChnittstelle einer horizontalen Linie und einer diagonalen

In [ ]:
def reshapeMatrix(corners):
    A = np.zeros((9, 9, 2), dtype=float)
    B = corners
    r,c = 1,1
    A[r:r+B.shape[0], c:c+B.shape[1]] += B
    corners = A
    return corners

In [ ]:
def line(p1, p2): # basis line transformation
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2): # calculate point of intersection, i can only be one and it has to be in range of picture
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y # x and y coordinates of intersection
    else:
        return False

In [ ]:
def drawLine(image, line): #debug method to show horizontal and diagonal lines

    color = [0,0,255] # color hoz lines red
    cv2.line(image, (image.shape[1]-1,line[1]),(0,line[0]), color=color, thickness=1)
    return image

In [ ]:
def getLine(array):
    [vx, vy, x, y] = cv2.fitLine(np.array(array),cv2.DIST_L2, 0, 0.01, 0.01)
    lefty = int((-x*vy/vx) + y)
    righty = int(((image.shape[1]-x)*vy/vx)+y)
    return lefty, righty

In [ ]:
def getDiagonal(corners, first, second): # diagonal always calcualted from top to bottom points
    diagonal = []
    if first[0] < second[0] and first[1] > second[1]: #diagonal right to left
        column = first[1]
        for x in range(first[0], second[0]+1):
            diagonal.append(corners[x][column])
            column = column - 1

    elif first[0] < second[0] and first[1] < second[1]: #diagonal left to right
        column = first[1]
        for x in range(first[0], second[0]+1):
            diagonal.append(corners[x][column])
            column = column + 1
    return diagonal

In [ ]:
def getHorizontal(corners, row, begin=0, end = 6): #inner horizontal points
    horizontalPoints = []
    for x in range(begin, end):
        horizontalPoints.append(corners[row][x])
    return horizontalPoints


In [ ]:
def getVertical(corners, column, begin=1, end=7):
    verticalPoints = []
    for x in range(begin, end):
        verticalPoints.append(corners[x][column])
    return verticalPoints

In [ ]:
def getIntersection(horizontal, diagonal):
    L1 = line((image.shape[1]-1,horizontal[1]),(0,horizontal[0]))
    L2 = line((image.shape[1]-1,diagonal[1]),(0,diagonal[0]))
    R = intersection(L1, L2)
    return R

In [ ]:
def getHorizontalOuterPoint(corners, image,  row, lineBegin, lineEnd):
    horizontal = getLine(getHorizontal(corners,row))
    diagonal = getLine(getDiagonal(corners,lineBegin,lineEnd))
    P = getIntersection(horizontal, diagonal)
    image = drawPoint(image, P)
    return image, P

In [ ]:
def getVerticalOuterPoint(corners, image,  column, lineBegin, lineEnd):
    vertical = getLine(getVertical(corners,column))
    diagonal = getLine(getDiagonal(corners,lineBegin,lineEnd))
    P = getIntersection(vertical, diagonal)
    image = drawPoint(image, P)
    return image, P

In [ ]:
def drawPoint(image, P):
    P = cv2.circle(image, (int(P[0]),int(P[1])) , radius=2, color=(0, 255, 0), thickness=-1)
    return image

In [ ]:
def getOuterPoints(corners, image):
    chessMatrix = reshapeMatrix(corners)
    # diagonal is read from top to bottom
    #######
    # Left side
    #######
    for x in range(0,7):
        if x < 4:
            image, chessMatrix[x+1][0] = getHorizontalOuterPoint(corners,image,x,(x+1,0),(6,5 - x))
        else:
            image, chessMatrix[x+1][0] = getHorizontalOuterPoint(corners,image,x,(0,x-1),(x-1,0))

    #######
    # Right side
    #######
    for x in range(0,7):
        if x < 4:
            image, chessMatrix[x+1][8] = getHorizontalOuterPoint(corners,image,x,(x+1,6),(6,5 - x))
        else:
            image, chessMatrix[x+1][8] = getHorizontalOuterPoint(corners,image,x,(0,7-x),(x-1,6))

    #######
    # Top
    #######
    for x in range(0,7):
        if x < 4:
            image, chessMatrix[0][x+1] = getVerticalOuterPoint(corners,image,x,(0,x+1),(5-x,6))
        else:
            image, chessMatrix[0][x+1] = getVerticalOuterPoint(corners,image,x,(0,x-1),(x-1,0))

    #######
    # Bottom
    #######
    for x in range(0,7):
        if x < 4:
            image, chessMatrix[8][x+1] = getVerticalOuterPoint(corners,image,x,(x+1,6),(6,x+1))
        else:
            image, chessMatrix[8][x+1] = getVerticalOuterPoint(corners,image,x,(7-x,0),(6,x-1))

    #print(chessMatrix)

    #######
    # Corner Points
    #######
    top = getLine(getHorizontal(chessMatrix,0,1,7))
    left = getLine(getVertical(chessMatrix,0,1,7))
    right = getLine(getVertical(chessMatrix,8,1,7))
    bottom = getLine(getHorizontal(chessMatrix,8,1,7))

    P = getIntersection(top, left)
    image = drawPoint(image, P)
    chessMatrix[0][0] = P
    P = getIntersection(top, right)
    image = drawPoint(image, P)
    chessMatrix[0][8] = P
    P = getIntersection(bottom, right)
    image = drawPoint(image, P)
    chessMatrix[8][8] = P
    P = getIntersection(bottom, left)
    image = drawPoint(image, P)
    chessMatrix[8][0] = P

    return image # later return matrix with new points

In [ ]:
import glob
import cv2
import numpy as np
from scipy.optimize import curve_fit
chess_images = glob.glob('../testFiles/emptyChessBoard/*.jpg')
# Select any index to grab an image from the list
for i, img in enumerate(chess_images):
    image = cv2.imread(img)
    im_bw = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    (found, corners) = cv2.findChessboardCorners(im_bw, (7, 7), None)
    if found:
        # Draw and display the corners
        cv2.drawChessboardCorners(image, (7, 7), corners, found)
        # Reshape matrix
        corners = np.reshape(corners,(7,7,2))

        image = getOuterPoints(corners, image)
    result_name = 'result/ChessBoard'+str(i)+'.jpg'
    cv2.imwrite(result_name, image)